In [1]:
import dotenv
%load_ext dotenv
%dotenv
import nest_asyncio
nest_asyncio.apply()

In [2]:
papers = [
    "./datasets/lora_paper.pdf", 
    "./datasets/longlora_efficient_fine_tuning.pdf"
]

In [3]:
from utils import create_doc_tools
from pathlib import Path

In [4]:
paper_to_tools_dict = {}

for paper in papers:
    print(f"Creating {paper} tool")
    path = Path(paper)
    vector_tool, summary_tool = await create_doc_tools(doc_name=path.stem, document_fp=path)
    paper_to_tools_dict[path.stem] = [vector_tool, summary_tool]

Creating ./datasets/lora_paper.pdf tool
Creating ./datasets/longlora_efficient_fine_tuning.pdf tool


In [5]:
paper_to_tools_dict

{'lora_paper': [<llama_index.core.tools.query_engine.QueryEngineTool at 0x1377db770>,
 'longlora_efficient_fine_tuning': [<llama_index.core.tools.query_engine.QueryEngineTool at 0x137b85850>,
  <llama_index.core.tools.query_engine.QueryEngineTool at 0x144ae3b90>]}

In [6]:
initial_tools = [t for paper in papers for t in paper_to_tools_dict[Path(paper).stem]]
print(initial_tools)

[<llama_index.core.tools.query_engine.QueryEngineTool object at 0x1377db770>, <llama_index.core.tools.query_engine.QueryEngineTool object at 0x127d42ea0>, <llama_index.core.tools.query_engine.QueryEngineTool object at 0x137b85850>, <llama_index.core.tools.query_engine.QueryEngineTool object at 0x144ae3b90>]


In [7]:
len(initial_tools)

4

#### AGENT WORKER

In [8]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [12]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools,
    llm=llm,
    verbose=True
)

agent = AgentRunner(agent_worker)

In [13]:
response = agent.query(
    "Explain what is Lora and how I can use it in my applications and saves me money?"
    "Explain what is LongLora and why it's being used."
    "Compare LongLora and LoRA. Which one is better?"
)

print(str(response))

Added user message to memory: Explain what is Lora and how I can use it in my applications and saves me money?Explain what is LongLora and why it's being used.Compare LongLora and LoRA. Which one is better?
=== Calling Function ===
Calling function: lora_paper_summary_query_engine_tool with args: {"input": "Explain what is Lora and how I can use it in my applications and saves me money?"}
=== Function Output ===
LoRA, or Low-Rank Adaptation, is a method used to adapt large-scale pre-trained language models to specific tasks or domains by injecting trainable rank decomposition matrices into each layer of the Transformer architecture. By implementing LoRA in your applications, you can efficiently switch between different tasks by replacing the matrices A and B, reducing storage requirements and task-switching overhead significantly. This allows you to share a pre-trained model and build multiple small LoRA modules for various tasks, reducing the need to store and load independent instanc